# NBA Analytics 360

## Importing necessary libraries:

In [7]:
import pandas as pd
import requests
import json

import time

import psycopg2
import psycopg2.extras as extras

## Data Collection from BALLDONTLIE API : 

### main function for data fetching:

In [8]:
def fetch_data_with_counter(data_type, url="https://www.balldontlie.io/api/v1", per_page=100, counter=None, last_counter=None):
    data = []
    requests_made = 0
    start_time = time.time()
    #loading value of final_counter to continue extraction from where we left:
    with open("final_counter.txt",'r') as f : 
        file_content = json.load(f)
    if counter is None:
            final_counter = (file_content[f"{data_type}_final_counter"])
            counter = final_counter + 1

    if last_counter is None:
        #while loop fetching all data available : 
        while type(counter) is int : 
            # GET request to the API with the specified URL and parameters stored in "call" variable :
            call = requests.get(f"{url}/{data_type}?per_page={per_page}&page={counter}")
            requests_made += 1
            # Extend the "data" list with retrieved data from API
            data.extend(call.json()["data"])

            # update "final_counter" if the "next_page" value is not an integer and the "data" list is not empty
            if call.json()["meta"]["next_page"] is not int and len(data)>0:
                final_counter = counter

            counter = call.json()["meta"]["next_page"]
            if requests_made == 60:
                print(len(games_data))
                elapsed_time = time.time() - start_time
                if elapsed_time < 60:
                    time.sleep(60 - elapsed_time)
                start_time = time.time()
                requests_made = 0
    else : 
        #while loop fetching all data available : 
        while counter<=last_counter : 
            # GET request to the API with the specified URL and parameters stored in "call" variable :
            call = requests.get(f"{url}/{data_type}?per_page={per_page}&page={counter}")
            # Extend the "data" list with retrieved data from the API
            data.extend(call.json()["data"])

            # update "final_counter" if the "next_page" value is not an integer and the "data" list is not empty
            if call.json()["meta"]["next_page"] is not int and len(data)>0:
                final_counter = counter

            counter = call.json()["meta"]["next_page"]
            
    file_content[f"{data_type}_final_counter"] = final_counter

    # update final_counter_file:
    with open("final_counter.txt", "w") as f: 
        json.dump(file_content, f)
    
    # turn json object into data_frame :
    df = pd.json_normalize(data)
    meta = call.json()["meta"]
    return df, meta

### PLAYERS DATA :

In [10]:
# players_df = fetch_data_with_counter(data_type="players")[0]
# players_df = players_df[["id", "first_name" , "last_name", "position", "height_feet", "height_inches", "weight_pounds", "team.id", "team.abbreviation", "team.city", "team.conference", "team.division", "team.name", "team.full_name"]]
players_df = pd.read_csv("api_data/player_data.csv", sep=";")

### TEAMS DATA :

In [11]:
#teams_df = fetch_data_with_counter(data_type="teams")[0]
teams_df = pd.read_csv("api_data/teams_data.csv", sep=";")

### GAMES DATA :

In [12]:
#games_df = fetch_data_with_counter(data_type="games")[0]
games_df = pd.read_csv("api_data/games_data.csv", sep=";")

### STATS DATA :

In [13]:
#stats_df = fetch_data_with_counter(data_type="stats")[0]
stats_df = pd.read_csv("api_data/stats_data.csv", sep=";")

## Connecting to Postgres DataBase : 

### Inserting Data Into PLAYERS Table :

In [88]:
connection, cursor = None, None

tuples = [tuple(x) for x in players_df.to_numpy()]

cols = ",".join(players_df.columns.to_list()).replace(".", "_")

table_name = "players"

try : 
    connection = psycopg2.connect(user = "postgres", 
                                  password="postgrepass", 
                                  host="localhost", 
                                  port="5432", 
                                  database="nba")
    cursor = connection.cursor()

    create_table_script = f''' CREATE TABLE IF NOT EXISTS {table_name}(
                            id integer,
                            first_name varchar,
                            last_name varchar,
                            position varchar,
                            height_feet float,
                            height_inches float,
                            weight_pounds float,
                            team_id integer,
                            team_abbreviation varchar,
                            team_city varchar,
                            team_conference varchar,
                            team_division varchar,
                            team_name varchar,
                            team_full_name varchar
                            )'''
    
    cursor.execute(create_table_script)
    
    insert_data_script = f''' INSERT INTO players ({cols}) VALUES %s ON CONFLICT DO NOTHING ''' 
     
    # Execute the query with the data using execute_values() method
    extras.execute_values(cursor, insert_data_script, tuples)
    
    connection.commit()
    
except Exception as error:
    print(error)
finally :
    if cursor is not None: 
        cursor.close()
    if connection is not None: 
        connection.close()



### Inserting Data Into Teams Table :

In [19]:
connection, cursor = None, None

tuples = [tuple(x) for x in teams_df.to_numpy()]

cols = ",".join(teams_df.columns.to_list()).replace(".", "_")

table_name = "teams"

try : 
    connection = psycopg2.connect(user = "postgres", 
                                  password="postgrepass", 
                                  host="localhost", 
                                  port="5432", 
                                  database="nba")
    cursor = connection.cursor()

    create_table_script = f''' CREATE TABLE IF NOT EXISTS {table_name}(
                            id int,
                            abbreviation varchar,
                            city varchar,
                            conference varchar,
                            division varchar,
                            full_name varchar,
                            name varchar
                            )'''
    
    cursor.execute(create_table_script)
    
    insert_data_script = f"INSERT INTO {table_name} ({cols}) VALUES %s ON CONFLICT DO NOTHING"
     
    # Execute the query with the data using execute_values() method
    extras.execute_values(cursor, insert_data_script, tuples)
    
    connection.commit()
    
except Exception as error:
    print(error)
finally :
    if cursor is not None: 
        cursor.close()
    if connection is not None: 
        connection.close()



### Inserting Data Into Games Table :

In [30]:
connection, cursor = None, None

tuples = [tuple(x) for x in games_df.to_numpy()]

cols = ",".join(games_df.columns.to_list()).replace(".", "_")

table_name = "games"

try : 
    connection = psycopg2.connect(user = "postgres", 
                                  password="postgrepass", 
                                  host="localhost", 
                                  port="5432", 
                                  database="nba")
    cursor = connection.cursor()

    create_table_script = f''' CREATE TABLE IF NOT EXISTS {table_name}(
                            id int,
                            date varchar,
                            home_team_score int,
                            period int,
                            postseason boolean,
                            season int,
                            status varchar,
                            time varchar,
                            visitor_team_score int,
                            home_team_id int,
                            home_team_abbreviation varchar,
                            home_team_city varchar,
                            home_team_conference varchar,
                            home_team_division varchar,
                            home_team_full_name varchar,
                            home_team_name varchar,
                            visitor_team_id int,
                            visitor_team_abbreviation varchar,
                            visitor_team_city varchar,
                            visitor_team_conference varchar,
                            visitor_team_division varchar,
                            visitor_team_full_name varchar,
                            visitor_team_name varchar
                            )'''
    
    cursor.execute(create_table_script)
    
    insert_data_script = f"INSERT INTO {table_name} ({cols}) VALUES %s ON CONFLICT DO NOTHING"
     
    # Execute the query with the data using execute_values() method
    extras.execute_values(cursor, insert_data_script, tuples)
    
    connection.commit()
    
except Exception as error:
    print(error)
finally :
    if cursor is not None: 
        cursor.close()
    if connection is not None: 
        connection.close()



### Inserting Data Into Stats Table :

In [32]:
connection, cursor = None, None

tuples = [tuple(x) for x in stats_df.to_numpy()]

cols = ",".join(stats_df.columns.to_list()).replace(".", "_")

table_name = "stats"

try : 
    connection = psycopg2.connect(user = "postgres", 
                                  password="postgrepass", 
                                  host="localhost", 
                                  port="5432", 
                                  database="nba")
    cursor = connection.cursor()

    create_table_script = f''' CREATE TABLE IF NOT EXISTS {table_name}(
                            id int,
                            ast real,
                            blk real,
                            dreb real,
                            fg3_pct real,
                            fg3a real,
                            fg3m real,
                            fg_pct real,
                            fga real,
                            fgm real,
                            ft_pct real,
                            fta real,
                            ftm real,
                            min varchar,
                            oreb real,
                            pf real,
                            pts real,
                            reb real,
                            stl real,
                            turnover real,
                            game_id int,
                            game_date varchar,
                            game_home_team_id int,
                            game_home_team_score int,
                            game_period int,
                            game_postseason boolean,
                            game_season int,
                            game_status varchar,
                            game_time varchar,
                            game_visitor_team_id int,
                            game_visitor_team_score int,
                            player_id real,
                            player_first_name varchar,
                            player_height_feet real,
                            player_height_inches real,
                            player_last_name varchar,
                            player_position varchar,
                            player_team_id real,
                            player_weight_pounds real,
                            team_id int,
                            team_abbreviation varchar,
                            team_city varchar,
                            team_conference varchar,
                            team_division varchar,
                            team_full_name varchar,
                            team_name varchar,
                            player real
                            )'''
    
    cursor.execute(create_table_script)
    
    insert_data_script = f"INSERT INTO {table_name} ({cols}) VALUES %s ON CONFLICT DO NOTHING"
     
    # Execute the query with the data using execute_values() method
    extras.execute_values(cursor, insert_data_script, tuples)
    
    connection.commit()
    
except Exception as error:
    print(error)
finally :
    if cursor is not None: 
        cursor.close()
    if connection is not None: 
        connection.close()